In [1]:
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader,DirectoryLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_astradb import AstraDBVectorStore

In [2]:
from src.helper import extract_pdf
document=extract_pdf("C:/GENAI/END-TO-END-MEDICAL-CHATBOT/data")


In [3]:
from src.helper import splitting
splitted_doc=splitting(document)

In [4]:
from src.helper import download_hugging_face_embeddings
embeddings=download_hugging_face_embeddings()

c:\Users\anil.nagamunthala\anaconda3\envs\medenv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [5]:
sample_vectors=embeddings.embed_query('how are u')
len(sample_vectors)

384

In [6]:
from langchain_astradb import AstraDBVectorStore
import os
astra_token=os.getenv('ASTRADB_TOKEN')
astradb_endpoint=os.getenv('ASTRADB_API_ENDPOINT')
vector_store = AstraDBVectorStore(
    collection_name="astra_vector_langchain",
    embedding=embeddings,
    api_endpoint=astradb_endpoint,
    token=astra_token,
)

In [7]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(splitted_doc))]

vector_store.add_documents(documents=splitted_doc, ids=uuids)

['69b96f06-6b0f-4727-88b8-b528c217c7a7',
 '328fefc3-f0a9-4607-a36e-19f623c9845f',
 '6cbdcaec-c38e-47d9-9959-7feb805204bc',
 'ff48cd2c-cbd2-4fee-881e-b972205baa1f',
 'fb1eb11c-0111-4cfc-8dc2-29f5cb41cf91',
 'd94e3963-8f27-461d-8cde-e381389fd0e7',
 'e51cb36a-737b-4f98-83c4-76e0d38df63a',
 'd84e7551-b1b0-4b5d-aed2-51e77a4cac5d',
 'f95ab4a5-f542-4f04-80dc-fa620512392f',
 'ee7445da-505a-4d68-bf27-ecea2de2dfa0',
 '34834f95-071d-475e-b393-cd61fa926b42',
 'fdda6e06-469e-4f73-adf4-dcefffd8a7cf',
 '20641d72-e049-4fe1-8966-955b532b747f',
 'a550f046-983e-4298-a4c1-f275178c6dda',
 '6288a978-f6b6-4afd-b6c5-001b52fdd6a7',
 '31b17bf5-fb80-4161-9fcd-7276d9a07bd9',
 'f1d6250c-ee4d-4da1-9bb8-70d20c2e58ce',
 '5d0480a3-1923-4633-84de-fe27d0372703',
 '54f70f84-d8e7-4578-bcb6-1a7662083bb2',
 '5e64e168-8fb4-4205-93c1-293406bdaace',
 'f4ce106c-1dbb-41c1-8801-387cd7b8528e',
 '98b2d989-2992-4070-b960-7b62ac2296f0',
 'f718156b-4bf7-4c11-8e76-f52e3cdc233e',
 '1b1729c8-7d97-4c0a-954c-6e8cc2b0a378',
 'b2df86b2-77fb-

In [8]:
retriever=vector_store.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [9]:
retrieved_docs = retriever.invoke("What is Acne?")
retrieved_docs

[Document(id='2a9e4ba9-d921-4922-bfab-07081ff45094', metadata={'source': 'C:\\GENAI\\END-TO-END-MEDICAL-CHATBOT\\data\\Medical_book.pdf', 'page': 39}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='0c310ecc-8d01-4348-b6c2-34244e6d6eaf', metadata={'source': 'C:\\GENAI\\END-TO-END-MEDICAL-CHATBOT\\data\\Medical_book.pdf', 'page': 38}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(id='2158156d-591c-45c5-b764-98d9ef02dabc', metadata={'source': 'C:\\GENAI\\END-TO-END-MEDICAL-CHATBOT\\data\\Medical_book.pdf', 'page': 37}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nD

In [ ]:
from src.helper import llm_model
api_key=os.getenv('GROQ_API_KEY')
llm=llm_model(api_key,0)

In [12]:
from src.prompt import prompt_template
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [14]:
input="What is Acne?"
result=qa.invoke({"query": input})
print("Response : ", result["result"])

Response :  Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria.
